## Global tracking for tfrGel10292018A 
- ### glass strain ramp steps ref;a-f
- ## Outline
  - Load steps a-f iteratively 
  - select only first and last frames
  - write small dataFrames for first/last with multiindex {step}_{first/last}
  - concatenate small df to get one large df
  - save and delete intermediate files

- what do i need out of this?
  1) for a given step, c for example, absolute position of the gel tracers
  2) Can i load all the gel steps at once? Probably, there is maaybe 200 frames and 2000 particles -> 4000k, which is
     far fewer than even a small time step. 
  3) If thats possible, why not run trackpy on all of it, right off the bat 
       - create stitched time series for all steps
           - with tp.PandasHDFStoreBig(globalStitchedFile) as s:
                 for data in loadStitched(__): s.put(data) # this will assign a key in the pyTables that corresponds to the index of the generate in loadStitched
           - ie I think this means that the 27th call to the generator, regardless of step or frame or anything else will assigned key frame=27, and you can access using s.get(27)
       - create simple multiindex that takes stack number and converts to step and frame number
       - run tracking
       - unstack output dataframe to recover three level multiindex step/frame/particle_id
       - with step sequence (reference, a, b, c, d, e, f)
       - varying nubmer of frames for each step with extremes at reference has 1 frame and f has 90
       - and particle ids should be unique
       - output complete trajectories in xyz with column for which step its currnetly in, then map discrete colormap to that column in ovito.
       - visualize jumps and drift in position with ovito 
  4) This is most similar to particleLocating/stitchTrack_combined.py, with the proviso that I only need to run on gel locations
     and that it is not currently implemented to stitch across experiments (with multiple yaml files.)


In [66]:
# load some modules, etc
import sys
sys.path.append('/Users/zsolt/Colloid_git/TractionRheoscopy')
from data_analysis.static import loadStitched
from particleLocating import postLocating as pl
from particleLocating import locationStitch as ls
from particleLocating import dplHash_v2 as dpl
import data_analysis.static as da

import trackpy as tp
import pandas as pd
import os

from importlib import reload

In [21]:
path ='/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018{step}/gel_stitched.h5'
path.format(step='a')

'/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018a/gel_stitched.h5'

In [40]:
for frame, data in enumerate(da.loadData2Mem(path.format(step='a')).unstack().T): 
    if frame==1: print(data)

1


In [41]:
gel_pos={}
gel_pos['a']= da.loadData2Mem(path.format(step='a'))
gel_pos['b']= da.loadData2Mem(path.format(step='b'))

In [57]:
tmp = gel_pos['b']
tmp['frame_global'] =  tmp.index.get_level_values(0) + max(gel_pos['a'].index.get_level_values(0))
tmp

x (um, imageStack)  y (um, imageStack)  z (um, imageStack)  \
frame particle                                                               
0     0                  42.761141           10.720819            4.031579   
      1                   8.430212           29.677551            4.579608   
      2                  48.403004           20.123339            5.120165   
      3                  15.481126           37.729078            4.693357   
      4                  13.875849           15.975838            5.571937   
...                            ...                 ...                 ...   
21    7907              174.559161          194.867359           26.171144   
      7908              232.227452          230.485989           21.974582   
      7909              204.325479          228.023236           28.123040   
      7910              195.545074          222.996431           29.457100   
      7911              205.214315          210.322420           34.284309   

                   x_std     y_std     z_std  frame_global  
frame particle                                              
0     0         0.101962  0.099703  0.069393            21  
      1         0.119339  0.118863  0.099681            21  
      2         0.119273  0.123181  0.126300            21  
      3         0.107810  0.102401  0.073096            21  
      4         0.094039  0.094813  0.074953            21  
...                  ...       ...       ...           ...  
21    7907      0.108122       NaN  0.101762            42  
      7908           NaN       NaN       NaN            42  
      7909      2.360549  2.365774  1.409450            42  
      7910      0.117607  0.122286  0.195274            42  
      7911      0.056009  0.078324       NaN            42  

[87528 rows x 7 columns]

In [232]:
# what is the data structure that needs to fed into tp.link_iter_df ? 
step ='a'
globalPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x'
_ = {'fName_frmt': '/tfrGel10212018A_shearRun10292018{}'.format(step) + '_stitched_gel_t{:03}.h5',
     'path': '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018{}/locations'.format(step)}
stitched = globalPath + '/tmp.h5'
mIdx = []
with tp.PandasHDFStoreBig(stitched) as s:
    # loop over steps
    for frame, data in enumerate(da.loadStitched([x for x in range(3)], **_)): 
        s.put(data)
        mIdx.append((step, frame))
        #print(data.keys())
with tp.PandasHDFStoreBig(stitched) as s: tmp= s.get(1)
tmp.describe().T

,count,mean,std,min,25%,50%,75%,max
"x (um, imageStack)",4044.0,1.167876e+02,6.591646e+01,2.463879,6.163981e+01,1.157326e+02,1.726873e+02,2.331316e+02
"y (um, imageStack)",4044.0,1.185771e+02,6.679654e+01,2.571643,6.107385e+01,1.189908e+02,1.760767e+02,2.332760e+02
"z (um, imageStack)",4044.0,2.049861e+01,7.973470e+00,3.555202,1.380499e+01,2.276674e+01,2.698082e+01,3.801794e+01
x_std,3957.0,1.052825e+02,5.967409e+03,0.040998,7.247490e-02,1.048032e-01,1.304211e-01,3.750526e+05
y_std,3955.0,1.034089e+02,6.076410e+03,0.014222,7.248682e-02,1.041719e-01,1.318512e-01,3.820736e+05
z_std,3827.0,4.587994e+00,8.522781e+01,0.022261,6.021840e-02,8.431548e-02,1.093605e-01,3.021085e+03
size_x,4044.0,1.068001e+01,1.287869e+00,6.000000,1.073356e+01,1.108813e+01,1.130111e+01,1.200000e+01
size_y,4044.0,1.068427e+01,1.300833e+00,6.000000,1.077030e+01,1.108647e+01,1.130419e+01,1.200000e+01
size_z,4044.0,8.631809e+00,1.499688e+00,4.000000,8.292711e+00,8.977196e+00,9.472167e+00,1.100000e+01
totalError,3768.0,7.961769e-01,1.699594e+01,0.007748,1.473965e-02,2.117080e-02,2.652468e-02,8.187910e+02


In [118]:
mIdx

[('a', 0), ('a', 1), ('a', 2)]

In [169]:
# not quite a multiindex, or if it is I dont know how this is stored internally.
# use s.get(frame) returns something similar if not identical to gel_pos.xs(0,level='frame')
# Differences:
#    - the pandas index is not labeled as 'particle'
#    - there is a frame column
#    - this data structure is unmodified by tp.link_df_iter. I think that method just lazily the indices across repeated calls s.get(t) and s.get(t+1)
#    - index at time t is unique (no duplicates) then these indices are linked. 
#.   - Whatever is unlinked in t+1 is assigned a new index that has is **unused** for across of the union of all indices in time 0 through t+1
s = tp.PandasHDFStoreBig(stitched)
s.get(0)

KeyError: 'No object named Frame_0 in the file'

In [107]:
# how to close tp pandas object outside a context manager? I think this is right and reasonable.
# note that two programs cannot both open a single hdf file simultaneously. So if this is not closed propoerly, you will get an hdf error eventually. 
# or at least that was a problem when running this in pyCharm. 
s.close()
del s

In [98]:
_ = {'fName_frmt': '/tfrGel09102018b_shearRun09232018a_stitched_gel_t{:03}.h5',
     'path': '/Users/zsolt/Colloid/DATA/tfrGel09102018b/shearRun09232018a/locations'}
tmp = pd.read_hdf(_['path'] + _['fName_frmt'].format(0))

tmp[tmp['keepBool']]['nonfluorescent_chunk']

0         2.42
1         0.13
2         4.44
3         0.05
4         0.21
         ...  
7829     21.54
7836    133.29
7844      3.87
7849      1.57
7850     16.45
Name: nonfluorescent_chunk, Length: 4715, dtype: float64

In [113]:
tmp[tmp['keepBool']].loc[0];

In [114]:
_ = {'fName_frmt': '/tfrGel09102018b_shearRun09232018a_stitched_gel_t{:03}.h5',
     'path': '/Users/zsolt/Colloid/DATA/tfrGel09102018b/shearRun09232018a/locations'}
for data in da.loadStitched([x for x in range(1)],'gel', **_): tmp = data
#tmp[['sed_Colloid_core', 'sed_Colloid_shell']]
tmp;

Stitched time 0


In [86]:
['a'] + ['b']

['a', 'b']

In [55]:
tmp.index.get_level_values(0)

Int64Index([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            ...
            21, 21, 21, 21, 21, 21, 21, 21, 21, 21],
           dtype='int64', name='frame', length=88750)

In [67]:
reload(da)

<module 'data_analysis.static' from '/Users/zsolt/Colloid_git/TractionRheoscopy/data_analysis/static.py'>

Stitched time 0


,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x_std,y_std,z_std,size_x,size_y,size_z,totalError,...,fluorescent_chunk_shell,gel_Tracer_core,gel_Tracer_shell,sed_Background_core,sed_Background_shell,nonfluorescent_chunk_core,nonfluorescent_chunk_shell,gel_Background_core,gel_Background_shell,frame
0,16.300632,43.228516,4.042353,0.078905,0.076069,0.054876,7.202560,6.937923,5.011305,0.015054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,4.686725,15.535923,4.543279,0.067140,0.066610,0.056463,11.442998,11.412003,9.334145,0.013785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,9.031294,45.091067,4.326189,204.272021,101.606319,7.313241,11.178836,10.083756,4.000000,26.259790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,40.402971,31.174276,5.585025,0.081011,0.081864,0.076182,11.774329,11.830373,10.086120,0.017493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,34.379511,13.603481,5.872966,0.065960,0.066070,0.056833,11.099806,11.158968,9.357399,0.013709,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7829,207.973356,210.585605,28.805414,0.179691,0.181787,NaN,8.700705,8.670713,11.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7836,201.060179,223.893960,16.042807,NaN,11.821719,2.550146,12.000000,10.467516,4.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7844,201.144040,200.678920,26.681270,0.186010,0.198107,NaN,9.534444,10.059968,7.230904,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7849,200.534640,225.997598,28.096634,0.105734,0.105966,0.070858,11.641857,11.620142,8.091006,0.020232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [186]:
pd.concat([gel_pos[step] for step in ['a','b']]).index
reload(da)

<module 'data_analysis.static' from '/Users/zsolt/Colloid_git/TractionRheoscopy/data_analysis/static.py'>

In [ ]:
# quick pseudo code
global_stitch = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10292018A_gel_global.h5'
max_disp=2 # probably need a large max displacement as the sample may drift betwewen steps. 
mIdx_tuples = []
tMax_list = []
metaPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018{step}/tfrGel10212018A_shearRun10292018{step}_metaData.yaml'

# create the single large stitched h5 file to mimic locating all the gel regions all at once, across experiments
with tp.PandasHDFStoreBig(global_stitch,'w') as s: # one file to rule them all, also not the force overwrite by calling with 'w' flag
    #for step in ['a','b']: # for the moment skip ref as that hasnt been stitched, nor has directory structure been determined. 
    for step in ['ref','a','b','c','d','e','f']: # all the steps, in order
        
        # open correspodning yaml file to find out time steps
        _ = dpl.dplHash(metaPath.format(step=step))
        metaData, hash_df =_.metaData, _.hash_df # not sure if I need all the metaData or just the hash_df
        del _ 

        # open corresponding stitched file
        _ = {'fName_frmt': '/tfrGel10212018A_shearRun10292018{}'.format(step) + '_stitched_gel_t{:03}.h5',
             'path': '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018{}/locations'.format(step)} 
        tMax = hash_df['t'].max() + 1
        offset = sum(tMax_list) # note the edge case sum([]) = 0 works by default
        for frame, data in enumerate(da.loadStitched(range(tMax), **_)):
            data['frame_local'] = data['frame']
            data['frame'] = data['frame'] + offset # i think the frame number needs to be incremented here to prevent tp from overwriting the same frame number at different steps
            data['step'] = step
            s.put(data)
            mIdx_tuples.append((step, frame))
        tMax_list.append(tMax) # increment now, after looping. Also, I explicitly checked off by one errors here.
        
        #for frame, data in enumerate(da.loadData2Mem(stitchedPath)): # this counts columns...not frames each frame, however many of them there are, in the current step
            #s.put(data) # this will put **all** time steps into a single large dataFrame, it would be great if 
            # somehow make the multiIndex that I want to generate. 
            

stitch_param={'neighbor_strategy': 'KDTree', 
              'pos_columns':['{} (um, imageStack)'.format(x) for x in ['x', 'y', 'z']],
              't_column':'frame', 'adaptive_stop':0.1, 'adaptive_step':0.95}

with tp.PandasHDFStoreBig(global_stitch) as s: # now track
    for linked in tp.link_df_iter(s, max_disp, **stitch_param): s.put(linked)

In [213]:
with tp.PandasHDFStoreBig(global_stitch) as s: 
    tmpa0 = s.get(0)
    tmpaf = s.get(21)
    tmpb0 = s.get(22)
    tmpbf = s.get(43)


In [227]:
pd.Index(tmpa0['particle'].values)
#pd.Index(tmpa0['particle'].values).intersection(pd.Index(tmpaf['particle'].values))
#pd.Index(tmpb0['particle'].values).intersection(pd.Index(tmpaf['particle'].values))
#pd.Index(tmpa0['particle'].values).intersection(pd.Index(tmpbf['particle'].values))
# load and reindex     

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3984, 3985, 3986, 3987, 3988, 3989, 3990, 3991, 3992, 3993],
           dtype='int64', length=3994)

In [125]:
metaPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018{step}/tfrGel10212018A_shearRun10292018{step}_metaData.yaml'
metaPath.format(step='a')
dpl.dplHash(metaPath.format(step=step)).hash_df['t'].max()

21

In [138]:
sum([])

0

In [2]:
import pandas as pd

In [9]:
for n,val in enumerate([1,2,3,4]): print(n,val)

0 1
1 2
2 3
3 4


In [7]:
pd.MultiIndex.from_tuples(list(zip(['a']*5,range(5))))

MultiIndex([('a', 0),
            ('a', 1),
            ('a', 2),
            ('a', 3),
            ('a', 4)],
           )

In [244]:
p = '/Users/zsolt/Colloid/DATA/tfrGel09102018b/shearRun09232018a/locations//tfrGel09102018b_shearRun09232018a_stitched_gel_t{:03}.h5'.format(0)
pd.read_hdf(p,key='0').keys()

Index(['index', 'z (px, hash)', 'y (px, hash)', 'x (px, hash)', 'mass', 'size',
       'ecc', 'signal', 'raw_mass', 'ep', 'n_iteration', 'frame', 'cluster',
       'cluster_size', 'disc_size', 'size_x', 'size_y', 'size_z', 'background',
       'background_std', 'signal_std', 'z_std', 'y_std', 'x_std', 'cost',
       'gel_Background_core', 'gel_Tracer_core', 'sed_Colloid_core',
       'sed_Background_core', 'fluorescent_chunk_core',
       'nonfluorescent_chunk_core', 'gel_Background_shell', 'gel_Tracer_shell',
       'sed_Colloid_shell', 'sed_Background_shell', 'fluorescent_chunk_shell',
       'nonfluorescent_chunk_shell', 'hashValue', 'material',
       'x (um, imageStack)', 'y (um, imageStack)', 'z (um, imageStack)',
       'totalError', 'keepBool', 'ep_z', 'ep_y', 'ep_x', 'size_z_std',
       'size_y_std', 'size_x_std', 'disc_size_std', 'gel_Background',
       'gel_Tracer', 'sed_Colloid', 'sed_Background', 'fluorescent_chunk',
       'nonfluorescent_chunk'],
      dtype='object')

In [245]:
pd.read_hdf(p,key='0').T

,0,1,2,3,4,5,6,7,8,9,...,7842,7843,7844,7845,7846,7847,7848,7849,7850,7851
index,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,0
"z (px, hash)",9.949021,13.288528,11.841258,20.2335,22.153108,21.565208,23.75518,29.145414,28.001014,35.284399,...,67.985766,98.181754,104.875131,107.915486,96.351126,102.75701,112.315486,114.310896,117.921418,27.552118
"y (px, hash)",365.900135,125.094983,382.096238,261.080662,108.291142,268.673809,194.864035,338.065838,82.90849,209.462212,...,400.700012,85.894076,137.034083,63.448371,46.105125,382.704854,323.367751,357.196501,81.990075,146.420953
"x (px, hash)",131.744628,30.754134,68.53299,341.330182,288.95227,398.947381,274.050983,98.470554,97.778308,384.007421,...,59.787702,37.256916,141.078607,277.230208,411.696767,357.322944,394.536765,135.779474,81.612106,342.190955
mass,23476072.59375,98822293.694792,15375995.064063,90010243.413021,91182461.357292,89997756.140365,84194543.936719,96595198.527083,10119844.329167,88707206.421875,...,307174.181403,12241969.745711,632744.40383,196930.280882,278190.49087,142006.84905,179990.041667,268132.223836,343569.226593,105347.112623
size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ecc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
signal,33360.92838,34575.126401,30920.863966,27381.675531,30305.363896,27565.497001,27313.304339,30080.884594,15075.011781,28500.522485,...,11950.478485,5035.051541,7486.347542,5808.456274,13994.09565,12922.23341,15468.280364,14065.784067,5986.027367,15717.515674
raw_mass,23944420.111689,99406876.576172,15794213.425233,90603770.999512,91785152.276794,90593962.083008,84786606.859761,97183825.43396,10499562.52691,89304318.620331,...,393914.236822,12619062.854331,699111.213228,259863.660342,356762.071043,194093.732292,238812.678114,344028.306354,418996.740464,152010.533618
ep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
pd.read_hdf(p,key='0').T

,0,1,2,3,4,5,6,7,8,9,...,1944497,1944498,1944499,1944500,1944501,1944502,1944503,1944504,1944505,1944506
index,0,1,2,3,4,5,6,7,8,9,...,1,2,3,4,5,6,7,8,9,10
"z (px, hash)",8.795004,11.173707,18.034293,18.533763,20.376743,21.352699,20.865412,26.758001,31.146351,46.380602,...,17.813894,22.692911,28.451237,30.124869,29.679165,30.762367,56.128241,62.785379,66.179516,68.522216
"y (px, hash)",264.993747,41.348529,358.219114,146.960013,44.119843,128.293419,231.271773,331.285718,346.124993,7.695277,...,323.287771,309.193986,304.282554,52.235846,64.900916,132.556342,167.362588,216.530102,390.813104,77.388915
"x (px, hash)",13.93174,285.43328,275.02788,205.651789,227.944071,316.325536,355.66122,221.393045,86.0154,102.034127,...,332.208044,392.918003,253.429223,353.879062,123.024732,101.105744,102.528188,33.020829,72.008799,178.451782
mass,189473.116912,3983513.801838,1073798.389338,209723.496507,827976.390074,2925563.535662,1070452.674449,803323.754044,2037188.187316,2087550.000919,...,2570459.538909,2570099.404473,3271101.084681,1682548.086275,1777803.644669,2855325.878002,2737922.051777,2784019.25962,1889625.387132,2671477.248284
size,3.229453,3.692949,3.005052,2.341535,2.88047,3.584732,3.247381,2.764781,3.404095,3.194602,...,3.043951,3.223923,3.314699,3.021346,3.070577,2.886788,3.01947,2.978449,2.919559,3.025226
ecc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
signal,2800.946071,15129.916113,10061.935324,4213.745083,8394.786819,12948.539793,9812.305486,10446.683179,11717.950388,14696.528171,...,22007.339323,21082.472516,21353.11227,20317.21921,17787.250397,27798.022013,23566.067824,23203.368751,21051.183897,22686.94043
raw_mass,213914.459384,4075808.795753,1130057.156552,231703.013942,878029.004813,3013323.709963,1128709.117252,849276.05051,2114381.578835,2162588.099283,...,2615102.888251,2609065.736293,3315165.417374,1712029.169722,1811614.31941,2899273.643443,2791029.667103,2837100.616845,1926989.934587,2719324.844703
ep,0.000089,0.000005,0.000017,0.000082,0.000022,0.000006,0.000017,0.000022,0.000009,0.000009,...,0.000015,0.000015,0.000012,0.000023,0.000022,0.000014,0.000014,0.000014,0.000021,0.000015


In [243]:
a,b = **{'a':1, 'b':2}

SyntaxError: invalid syntax (<ipython-input-243-5150054aff17>, line 1)

'a'